In [1]:
import requests
import json
import urllib.request
import pandas as pd
import ast

In [43]:
response_details_merge = pd.read_csv('../data/Data 6 - Merged Responses and Movie Details.csv', encoding='unicode_escape')
response_details_merge.head()

,id_x,person_mbti,mbti_test,character,movie,new_title,genre_ids,id_y,popularity,release_date,title,vote_average,vote_count
0,0,ENFP,michaelcaloz.com,Tony Stark,Avengers: Endgame,Avengers: Endgame,"[12, 878, 28]",299534,240.886,24/04/2019,Avengers: Endgame,8.255,24867
1,0,ENFP,michaelcaloz.com,Rocket Raccoon,Guardians of the Galaxy Vol. 3,Guardians of the Galaxy Vol. 3,"[878, 12, 28]",447365,169.896,03/05/2023,Guardians of the Galaxy Vol. 3,7.972,6262
2,0,ENFP,michaelcaloz.com,Captain America,Avengers: Endgame,Avengers: Endgame,"[12, 878, 28]",299534,240.886,24/04/2019,Avengers: Endgame,8.255,24867
3,0,ENFP,michaelcaloz.com,Peter Parker,Spider-Man: Far From Home,Spider-Man: Far From Home,"[28, 12, 878]",429617,124.720,28/06/2019,Spider-Man: Far From Home,7.400,15144
4,0,ENFP,michaelcaloz.com,Hobie Brown,Spider-Man: Across the Spider-Verse,Spider-Man: Across the Spider-Verse,"[16, 28, 12, 878]",569094,187.181,31/05/2023,Spider-Man: Across the Spider-Verse,8.366,6265


# Actions
### Columns to be removed
- `Unnamed: 0`
- `movie` and `title` since `new_title` was assigned to hold the actual movie title in Script 4
- `vote_count`
### Renames
- `id_x` as `person_id`
- `new_title` as `movie`
- `id_y` as `movie_id`
- `popularity` as `movie_popularity`
- `vote_average` as `movie_vote_avg`
### Others
- Change `release_date` to only capture year information as `movie_year`


In [51]:
response_details = response_details_merge.copy()

response_details.drop(columns = ['person_mbti', 'movie', 'title', 'vote_count'], inplace=True)
response_details.rename(columns = {'id_x':'person_id', 'new_title':'movie', 'id_y':'movie_id', 'popularity':'movie_popularity', 'vote_average':'movie_vote_avg'}, inplace = True)
response_details['movie_year'] = response_details_merge['release_date'].astype(str).str[:4]
response_details.drop(columns = ['release_date'], inplace=True)


retain_chars = 'a-zA-Z0-9\s\-:–\(\)'

response_details['character'] = response_details['character'].str.lower().replace(f'[^{retain_chars}]', '', regex=True)
response_details['movie'] = response_details['movie'].str.lower().replace(f'[^{retain_chars}]', '', regex=True)

response_details

,person_id,mbti_test,character,movie,genre_ids,movie_id,movie_popularity,movie_vote_avg,movie_year
0,0,michaelcaloz.com,tony stark,avengers: endgame,"[12, 878, 28]",299534,240.886,8.255,24/0
1,0,michaelcaloz.com,rocket raccoon,guardians of the galaxy vol 3,"[878, 12, 28]",447365,169.896,7.972,03/0
2,0,michaelcaloz.com,captain america,avengers: endgame,"[12, 878, 28]",299534,240.886,8.255,24/0
3,0,michaelcaloz.com,peter parker,spider-man: far from home,"[28, 12, 878]",429617,124.720,7.400,28/0
4,0,michaelcaloz.com,hobie brown,spider-man: across the spider-verse,"[16, 28, 12, 878]",569094,187.181,8.366,31/0
...,...,...,...,...,...,...,...,...,...
700,123,michaelcaloz.com,agent smith,the matrix,"[28, 878]",603,575.344,8.215,31/0
701,123,michaelcaloz.com,daniel plainview,there will be blood,[18],7345,36.996,8.084,26/1
702,123,michaelcaloz.com,eraserhead baby,eraserhead,"[14, 27]",985,67.775,7.400,03/0
703,123,michaelcaloz.com,patrick bateman,american psycho,"[53, 18, 80]",1359,84.339,7.404,13/0


In [52]:
char_mbti_df = pd.read_csv('../data/Data 8 - Franchise-expanded Movies from Data 2.csv', encoding='unicode_escape')

char_mbti_df['char'] = char_mbti_df['char'].str.lower().replace(f'[^{retain_chars}]', '', regex=True)
char_mbti_df['movie'] = char_mbti_df['movie'].str.lower().replace(f'[^{retain_chars}]', '', regex=True)

char_mbti_df


,char,movie,mbti,votes
0,guido anselmi,8 12,INTP,57
1,king leonidas of sparta,300,ENTJ,460
2,artemisia,300,ENTJ,127
3,xerxes,300,INTJ,103
4,gorgo,300,INFJ,62
...,...,...,...,...
10718,gazelle,zootopia (2016),ESFP,107
10719,mr manchas,zootopia (2016),ISFJ,40
10720,doug ramses,zootopia (2016),INTP,30
10721,travis,zootopia (2016),ESTP,27


In [46]:
zero = response_details.iloc[-1]
char = zero['character']
movie = zero['movie']
print(char, movie)

the duke fear and loathing in las vegas


In [47]:
char_mbti_df_copy = char_mbti_df.copy()
query_bool = char_mbti_df_copy['char'].str.contains(char) & char_mbti_df_copy['movie'].str.contains(movie)
# query_bool =  query_bool[query_bool.values==True]
char_mbti_df_copy = char_mbti_df_copy[query_bool]
char_mbti_df_copy
# print(query_bool)

,char,movie,mbti,votes


In [53]:
char_mbti_df_copy = char_mbti_df.copy()

failed_index = []
failed_char = []
failed_movie = []

k = len(response_details)
# k = 5

for i in range(k):
    zero = response_details.iloc[i]
    char = zero['character']
    movie = zero['movie']
    # print(char, movie)

    query_bool = char_mbti_df_copy['char'].str.contains(char) & char_mbti_df_copy['movie'].str.contains(movie)
    char_mbti_query = char_mbti_df_copy[query_bool]
    try:
        char_df_subset = char_mbti_query.iloc[0]
        # print(i, char_df_subset['char'], char_df_subset['mbti'])
    except:
        print(i, '      FAILED!  ', char,  '    ', movie)
        failed_index.append(i)
        failed_char.append(char)
        failed_movie.append(movie)

failed_df = pd.DataFrame({'failed_index': failed_index, 'failed_char': failed_char, 'failed_movie': failed_movie})
failed_df

23       FAILED!   mathilda      mathilda
34       FAILED!   yorichii       demon slayer: kimetsu no yaiba -to the hashira training-
44       FAILED!   jang okju      ballerina
46       FAILED!   jordan turner      the call
55       FAILED!   jiro horikoshi      the wind rises
56       FAILED!   gretta      begin again
57       FAILED!   sakura yamauchi      let me eat your pancreas
58       FAILED!   makoto konno      the girl who leapt through time
63       FAILED!   willy wonka       charlie and the chocolate factory
64       FAILED!   sherlock holmes       sherlock holmes
65       FAILED!   kirk lazarus       tropic thunder
66       FAILED!   alexander claremont-diaz      red white  royal blue
67       FAILED!   hannibal lecter       the silence of the lambs
68       FAILED!   hamish mycroft  merlin      kingsman: the secret service
72       FAILED!   capt manuel      seven sundays
73       FAILED!   gojo satori      jujutsu kaisen 0
74       FAILED!   amy dunne      gone girl
75  

C:\Users\Marco\AppData\Local\Temp\ipykernel_19784\4136469517.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  query_bool = char_mbti_df_copy['char'].str.contains(char) & char_mbti_df_copy['movie'].str.contains(movie)


224       FAILED!   professor langdon      the da vinci code
228       FAILED!   captain underpants      captain underpants: the first epic movie
232       FAILED!   yas      rye lane
233       FAILED!   alice      plus one
236       FAILED!   sasha tran      always be my maybe
244       FAILED!   dr emmett brown      back to the future
247       FAILED!   shun      from up on poppy hill
248       FAILED!   park ki joon      midnight runners
249       FAILED!   padre burgos      gomburza
250       FAILED!   howl      howls moving castle
252       FAILED!   howl      howls moving castle
253       FAILED!   rajun      3 idiots
254       FAILED!   sophie      howls moving castle
256       FAILED!   isabelle madrigal      encanto
258       FAILED!   gojo      jujutsu kaisen 0
259       FAILED!   mei mei      jujutsu kaisen 0
260       FAILED!   elle woods      legally blondes
273       FAILED!   elizabeth bennet      pride  prejudice
274       FAILED!   jane smith      mr-9: do or die
278 

,failed_index,failed_char,failed_movie
0,23,mathilda,mathilda
1,34,yorichii,demon slayer: kimetsu no yaiba -to the hashira...
2,44,jang okju,ballerina
3,46,jordan turner,the call
4,55,jiro horikoshi,the wind rises
...,...,...,...
262,696,kiri,avatar: the way of water
263,697,terminator,terminator 3: rise of the machines
264,699,uta,one piece film red
265,702,eraserhead baby,eraserhead
